# Startup

In [1]:
import krpc
import time
import numpy as np

In [2]:
conn = krpc.connect(name='Falcon 1')
vessel = conn.space_center.active_vessel
print(f'Connected to {vessel.name}')

Connected to Falcon 1


# Constants

In [3]:
g = conn.space_center.bodies['Kerbin'].surface_gravity

# Helper Functions

In [4]:
def find_payload_mass(vessel):
    '''Calculate the payload mass of the vessel, in tonnes.'''
    payload_decoupler = vessel.parts.with_tag('payload_decoupler')[0]
    return sum([child.mass for child in payload_decoupler.children])/1000.0
    

def calc_throttle_for_twr(target_twr, vessel):
    '''Calculate the required throttle (0.0-1.0) to
    meet the given TWR. If desired TWR is too high to
    be met, set throttle to 1.0.'''
    mass = vessel.mass
    weight = mass*g
    
    if vessel.available_thrust == 0.0:
        # safe against division by 0
        return 0.0
    else:
        throttle = target_twr*weight/vessel.available_thrust
        return min(1.0, throttle)
    
    
print(find_payload_mass(vessel))

0.13


# Ascent

## Ignition to MECO

1. Set target heading to straight up.

2. Set throttle such that thrust-to-weight ratio (TWR) is 1.3.

3. Ignite main engine.

4. Wait until vertical speed is 100 m/s.

5. Pitch over due east at a rate such that heading is at 45 degrees at an altitude of 15 km. The pitch rate is determined from the payload mass. The equation used to calculate the pitch rate $\dot{\theta}$, in degrees/second, was calculated empirically over several trials for a range of payload masses $m$ (tonnes):

$$\dot{\theta} = 0.0041m^2 + 0.0259m + 0.5801$$

6. Maintain a pitch of 45 degrees until the apoapsis altitude reaches 30 km. This will ensure the second stage can carry even the heaviest payload (2.25 tonnes) to orbit.

7. Set throttle to 0.0.

In [5]:
# set target heading to straight up
vessel.auto_pilot.target_pitch_and_heading(90.0, 90.0)
vessel.auto_pilot.engage()
time.sleep(1)

# set throttle s.t. TWR <= 1.3
target_twr = 1.3
vessel.control.throttle = calc_throttle_for_twr(target_twr, vessel)
vessel.control.activate_next_stage()

# go straight up until vert. speed is 100 m/s
while vessel.flight(vessel.orbit.body.reference_frame).vertical_speed < 100.0:
    vessel.control.throttle = calc_throttle_for_twr(target_twr, vessel)
    time.sleep(0.1)
    
# pitch over at a constant rate s.t. heading is 45 deg at alt. of 15 km
target_twr = 1.5
vessel.control.throttle = calc_throttle_for_twr(target_twr, vessel)
calc_pitch_rate = lambda m: 0.0041*m**2 + 0.0259*m + 0.5801
pitch_rate = calc_pitch_rate(find_payload_mass(vessel))
pitch_start = time.time()
while vessel.flight().pitch > 45.0:
    vessel.control.throttle = calc_throttle_for_twr(target_twr, vessel)
    target_pitch = 90.0 - (time.time() - pitch_start)*pitch_rate
    vessel.auto_pilot.target_pitch_and_heading(target_pitch, 90.0)
time.sleep(0.1)

# maintain pitch of 45 deg until apoapsis altitude reaches 30 km
while vessel.orbit.apoapsis_altitude < 30000:
    vessel.control.throttle = calc_throttle_for_twr(target_twr, vessel)
    vessel.auto_pilot.target_pitch_and_heading(45.0, 90.0)
time.sleep(0.1)

# MECO
vessel.control.throttle = 0.0
vessel.control.activate_next_stage()  # stage separation
time.sleep(0.1)

## MECO to SECO-1

In [6]:
# SES-1
vessel.control.activate_next_stage()  # second engine startup
pitch_rate = 1.0
pitch_start = time.time()

# pitch over until horizontal
while vessel.flight().pitch > 0.0:
    target_twr = 1.75
    vessel.control.throttle = calc_throttle_for_twr(target_twr, vessel)
    target_pitch = 45.0 - (time.time() - pitch_start)*pitch_rate
    vessel.auto_pilot.target_pitch_and_heading(target_pitch, 90.0)
time.sleep(0.1)

# continue burning horizontal until apoapsis altitude at 100 km
while vessel.orbit.apoapsis_altitude < 100000.0:
    target_twr = 1.75
    vessel.control.throttle = calc_throttle_for_twr(target_twr, vessel)
    vessel.auto_pilot.target_pitch_and_heading(0.0, 90.0)
time.sleep(0.1)

# SECO-1
vessel.control.throttle = 0.0

## SECO-1 to Circularization

def find_ascent_altitude(pitch_rate):
    # set target heading to straight up
    vessel.auto_pilot.target_pitch_and_heading(90.0, 90.0)
    vessel.auto_pilot.engage()
    time.sleep(1)

    # set throttle s.t. TWR <= 1.3
    target_twr = 1.3
    vessel.control.throttle = calc_throttle_for_twr(target_twr, vessel)
    vessel.control.activate_next_stage()

    # go straight up until vert. speed is 100 m/s
    while vessel.flight(vessel.orbit.body.reference_frame).vertical_speed < 100.0:
        vessel.control.throttle = calc_throttle_for_twr(target_twr, vessel)
        time.sleep(0.1)

    # pitch over at a constant rate s.t. heading is 45 deg at alt. of 15 km
    target_twr = 1.5
    vessel.control.throttle = calc_throttle_for_twr(target_twr, vessel)
    pitch_start = time.time()
    while vessel.flight().pitch > 45.0:
        vessel.control.throttle = calc_throttle_for_twr(target_twr, vessel)
        target_pitch = 90.0 - (time.time() - pitch_start)*pitch_rate
        vessel.auto_pilot.target_pitch_and_heading(target_pitch, 90.0)
    return vessel.flight().mean_altitude
    

saves = ['pad_125', 'pad_125', 'pad_000', 'pad_000']
for i, save in enumerate(saves):
    conn.space_center.load(save)
    vessel = conn.space_center.active_vessel
    pitch_rates = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    ascent_altitudes = []
    for pitch_rate in pitch_rates:
        conn.space_center.load(save)
        asc_alt = find_ascent_altitude(pitch_rate)
        ascent_altitudes.append(asc_alt)
    with open(f'{save}_{i%2}.txt', 'w') as f:
        for asc in ascent_altitudes:
            f.write(f'{asc}\n')
    print(f'Completed {i+1} of 6')